In [58]:
import cv2
import numpy as np
import tensorflow as tf

In [59]:
interpreter = tf.lite.Interpreter(model_path="dhito2-openmv\\trained.tflite")
interpreter.allocate_tensors()

In [60]:
interpreter.get_signature_list()

{'serving_default': {'inputs': ['x'], 'outputs': ['output_0']}}

In [61]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_x:0', 'index': 0, 'shape': array([ 1, 32, 32,  1]), 'shape_signature': array([ 1, 32, 32,  1]), 'dtype': <class 'numpy.int8'>, 'quantization': (0.003921568859368563, -128), 'quantization_parameters': {'scales': array([0.00392157], dtype=float32), 'zero_points': array([-128]), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 14, 'shape': array([1, 2]), 'shape_signature': array([1, 2]), 'dtype': <class 'numpy.int8'>, 'quantization': (0.00390625, -128), 'quantization_parameters': {'scales': array([0.00390625], dtype=float32), 'zero_points': array([-128]), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [62]:
with open("dhito2-openmv\labels.txt", "r") as f:
    labels = [line.strip() for line in f.readlines()]


In [63]:
scale, zero_point = input_details[0]['quantization']
print(scale)
print(zero_point)

0.003921568859368563
-128


In [64]:
def preprocess_frame(frame):

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    resized = cv2.resize(gray, (32, 32))

    normalized = resized / 255.0

    input_scale, input_zero_point = input_details[0]['quantization']
    quantized_input = (normalized / input_scale + input_zero_point).astype(np.int8)
    return quantized_input, resized

In [65]:
def classify_frame(frame):

    input_data, resized_frame = preprocess_frame(frame)

    input_data = np.expand_dims(input_data, axis=0)
    input_data = np.expand_dims(input_data, axis=-1)

    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])

    output_scale, output_zero_point = output_details[0]['quantization']
    predictions = (output_data.astype(np.float32) - output_zero_point) * output_scale

    print(f"Output data shape: {output_data.shape}")
    print(f"Raw output data: {output_data}")
    print(f"Dequantized output data: {predictions}")

    predictions = predictions[0]

    if predictions.shape[0] != len(labels):
        raise ValueError(f"Mismatch between output shape {predictions.shape} and number of labels {len(labels)}")

    label_probs = {labels[i]: predictions[i] for i in range(len(labels))}

    return label_probs, resized_frame

In [66]:
cap = cv2.VideoCapture(1)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    try:
        label_probs, resized_frame = classify_frame(frame)

        y_position = 30
        for label, prob in label_probs.items():
            cv2.putText(frame, f"{label}: {prob:.4f}", (10, y_position), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            y_position += 40

    except ValueError as e:
        print(f"Error: {e}")
        cv2.putText(frame, "Classification error", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', frame)
    cv2.imshow('Resized Frame', resized_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Output data shape: (1, 2)
Raw output data: [[-112  112]]
Dequantized output data: [[0.0625 0.9375]]
Output data shape: (1, 2)
Raw output data: [[-112  112]]
Dequantized output data: [[0.0625 0.9375]]
Output data shape: (1, 2)
Raw output data: [[-112  112]]
Dequantized output data: [[0.0625 0.9375]]
Output data shape: (1, 2)
Raw output data: [[-112  112]]
Dequantized output data: [[0.0625 0.9375]]
Output data shape: (1, 2)
Raw output data: [[-112  112]]
Dequantized output data: [[0.0625 0.9375]]
Output data shape: (1, 2)
Raw output data: [[-110  110]]
Dequantized output data: [[0.0703125 0.9296875]]
Output data shape: (1, 2)
Raw output data: [[-70  70]]
Dequantized output data: [[0.2265625 0.7734375]]
Output data shape: (1, 2)
Raw output data: [[-18  18]]
Dequantized output data: [[0.4296875 0.5703125]]
Output data shape: (1, 2)
Raw output data: [[-37  37]]
Dequantized output data: [[0.35546875 0.64453125]]
Output data shape: (1, 2)
Raw output data: [[-37  37]]
Dequantized output data: 